# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

# Описание проекта

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину. 
Шаги для выбора локации обычно такие:
В избранном регионе собирают характеристики для скважин: качество нефти и объём её запасов;
Строят модель для предсказания объёма запасов в новых скважинах;
Выбирают скважины с самыми высокими оценками значений;
Определяют регион с максимальной суммарной прибылью отобранных скважин.
Вам предоставлены пробы нефти в трёх регионах. Характеристики для каждой скважины в регионе уже известны. Постройте модель для определения региона, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой Bootstrap.

Условия задачи:
Для обучения модели подходит только линейная регрессия (остальные — недостаточно предсказуемые).
При разведке региона исследуют 500 точек, из которых с помощью машинного обучения выбирают 200 лучших для разработки.
Бюджет на разработку скважин в регионе — 10 млрд рублей.
При нынешних ценах один баррель сырья приносит 450 рублей дохода. Доход с каждой единицы продукта составляет 450 тыс. рублей, поскольку объём указан в тысячах баррелей.
После оценки рисков нужно оставить лишь те регионы, в которых вероятность убытков меньше 2.5%. Среди них выбирают регион с наибольшей средней прибылью.
Данные синтетические: детали контрактов и характеристики месторождений не разглашаются.

План проекта:
1. Необходимо загрузить и подготовить данные для анализа.
2. Обучим и проверим модель для каждого региона:\
 2.1. Разбить данные на обучающую и валидационную выборки в соотношении 75:25.\
 2.2. Обучить модель и сделать предсказания на валидационной выборке.\
 2.3. Сохранить предсказания и правильные ответы на валидационной выборке.\
 2.4. Напечатать на экране средний запас предсказанного сырья и RMSE модели.\
 2.5. Проанализировать результаты.
3. Подготовиться к расчёту прибыли:\
 3.1. Все ключевые значения для расчётов сохранить в отдельных переменных.\
 3.2. Рассчитать достаточный объём сырья для безубыточной разработки новой скважины. Сравнить полученный объём сырья со средним запасом в каждом регионе. \
 3.3. Сделать выводы по этапу подготовки расчёта прибыли.
4. Написать функцию для расчёта прибыли по выбранным скважинам и предсказаниям модели:\
 4.1. Выбрать скважины с максимальными значениями предсказаний. \
 4.2. Просуммировать целевое значение объёма сырья, соответствующее этим предсказаниям.\
 4.3. Рассчитать прибыль для полученного объёма сырья.
5. Посчитать риски и прибыль для каждого региона:\
 5.1. Применить технику Bootstrap с 1000 выборок, чтобы найти распределение прибыли.\
 5.2. Найти среднюю прибыль, 95%-й доверительный интервал и риск убытков. Убыток — это отрицательная прибыль.\
 5.3. Сделать общие выводы: предложите регион для разработки скважин и обоснуйте выбор.

## Загрузка и подготовка данных

### Импортируем необходимые библиотеки

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from scipy import stats as st
from numpy.random import RandomState

In [2]:
state = np.random.RandomState(12345)

### Прочитаем файлы данных

In [3]:
data_0 = pd.read_csv('datasets/geo_data_0.csv')
data_1 = pd.read_csv('datasets/geo_data_1.csv')
data_2 = pd.read_csv('datasets/geo_data_2.csv')

### Исследуем данные

In [4]:
print('Регион 0')
print(data_0.info())
print()
print('Регион 1')
print(data_1.info())
print()
print('Регион 2')
print(data_2.info())

Регион 0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None

Регион 1
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None

Регион 2
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (to

In [5]:
print('Регион 0', data_0.shape)
print()
print('Регион 1', data_1.shape)
print()
print('Регион 2', data_2.shape)

Регион 0 (100000, 5)

Регион 1 (100000, 5)

Регион 2 (100000, 5)


In [6]:
print('Регион 0')
display(data_0.head())

Регион 0


,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


In [7]:
print('Регион 1')
display(data_1.head())

Регион 1


,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


In [8]:
print('Регион 2')
display(data_2.head())

Регион 2


,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


In [9]:
print('Регион 0')
display(data_0.describe())
print()
print('Регион 1')
display(data_1.describe())
print()
print('Регион 2')
display(data_2.describe())

Регион 0


,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.500419,0.250143,2.502647,92.500000
std,0.871832,0.504433,3.248248,44.288691
min,-1.408605,-0.848218,-12.088328,0.000000
25%,-0.072580,-0.200881,0.287748,56.497507
50%,0.502360,0.250252,2.515969,91.849972
75%,1.073581,0.700646,4.715088,128.564089
max,2.362331,1.343769,16.003790,185.364347



Регион 1


,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,1.141296,-4.796579,2.494541,68.825000
std,8.965932,5.119872,1.703572,45.944423
min,-31.609576,-26.358598,-0.018144,0.000000
25%,-6.298551,-8.267985,1.000021,26.953261
50%,1.153055,-4.813172,2.011479,57.085625
75%,8.621015,-1.332816,3.999904,107.813044
max,29.421755,18.734063,5.019721,137.945408



Регион 2


,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.002023,-0.002081,2.495128,95.000000
std,1.732045,1.730417,3.473445,44.749921
min,-8.760004,-7.084020,-11.970335,0.000000
25%,-1.162288,-1.174820,0.130359,59.450441
50%,0.009424,-0.009482,2.484236,94.925613
75%,1.158535,1.163678,4.858794,130.595027
max,7.238262,7.844801,16.739402,190.029838


In [10]:
print('Регион 0')
print(data_0.duplicated().sum())
print()
print('Регион 1')
print(data_1.duplicated().sum())
print()
print('Регион 2')
print(data_2.duplicated().sum())

Регион 0
0

Регион 1
0

Регион 2
0


In [11]:
# # Проверим корреляцию между признаками

# display('Регион 0')
# display(data_0.corr())

In [12]:
# display('Регион 1')
# display(data_1.corr())

In [13]:
# display('Регион 2')
# display(data_2.corr())

В  Регионе 1 очень высокая корреляция между f2 и product. Признак f2 следует удалить.

In [14]:
# data_1 = data_1.drop(['f2'],axis = 1)

Вывод по результатам анализа данных:

1. Пропуски - отсутствуют
2. Типы данных - корректны
3. Названия столбцов - корректны
4. Дубликаты - отсутствуют
5. Объем запасов - положительный
6. Коррелирующие признаки - учтены

## Обучение и проверка модели

In [15]:
# Создадим функцию для расчета предсказанных средних запасов и RMSE

def model_score(data):
    features = data.drop(['id','product'], axis = 1)
    target = data['product']
    
    features_train, features_valid, target_train, target_valid = train_test_split(features, 
                                                                                  target, 
                                                                                  test_size=0.25, 
                                                                                  random_state=state)
    
    model = LinearRegression()
    model.fit(features_train, target_train)
    predictions_valid = model.predict(features_valid)
    
# Упорядочим индексы в таргете и предсказаниях
    predictions_valid = pd.Series(predictions_valid).reset_index(drop=True)
    target_valid = pd.Series(target_valid).reset_index(drop=True)
    
    average_stock = predictions_valid.mean()
    rmse = mean_squared_error(target_valid, predictions_valid)**0.5
    
    return average_stock, rmse, predictions_valid, target_valid

In [16]:
model_data_0 = model_score(data_0)
print('Регион 0')
print('Средний запас:', model_data_0[0])
print('RMSE модели:', model_data_0[1])

Регион 0
Средний запас: 92.59256778438035
RMSE модели: 37.5794217150813


In [17]:
model_data_1 = model_score(data_1)
print('Регион 1')
print('Средний запас:', model_data_1[0])
print('RMSE модели:', model_data_1[1])

Регион 1
Средний запас: 68.76995145799754
RMSE модели: 0.889736773768065


In [18]:
model_data_2 = model_score(data_2)
print('Регион 2')
print('Средний запас:', model_data_2[0])
print('RMSE модели:', model_data_2[1])

Регион 2
Средний запас: 95.087528122523
RMSE модели: 39.958042459521614


Выводы:\
Запасы в Регионе 0 и Регионе 2 приблизительно равны, ошибка также невелика.\
Запасы в Регионе 1 наименьшие, c учетом корреляции призака и таргета ошибка совсем невелика.

## Подготовка к расчёту прибыли

In [19]:
# Все ключевые значения для расчётов сохраняем в отдельных переменных.
budget_per_region = 10_000_000_000 # Бюджет на разработку скважин в регионе
research_points = 500 # Исследуемые при разработке региона точки
best_research_points = 200 # Выбранные машинным обучением точки для разработки
income_per_barrel = 450_000 # Доход с каждой единицы продукта
damage_threshold =  0.025 # Критическая вероятность убытков
break_even = (budget_per_region)/(income_per_barrel*best_research_points) # Достаточный объем добычи для безубыточной разработки

In [20]:
print('Достаточный объем добычи для безубыточной разработки', round(break_even), 'тыс.баррелей')

Достаточный объем добычи для безубыточной разработки 111 тыс.баррелей


In [21]:
break_even_count = data_0['product'][data_0['product'] > round(break_even)].count()
print('Количество скважин в Регионе 0, имеющих запас выше точки безубыточности:', 
      break_even_count)
print(f'Процентное соотношение скважин, имеющих запас выше точки безубыточности: {break_even_count / 100000 : .0%}')

Количество скважин в Регионе 0, имеющих запас выше точки безубыточности: 36676
Процентное соотношение скважин, имеющих запас выше точки безубыточности:  37%


In [22]:
break_even_count = data_1['product'][data_1['product'] > round(break_even)].count()
print('Количество скважин в Регионе 1, имеющих запас выше точки безубыточности:', 
      break_even_count)
print(f'Процентное соотношение скважин, имеющих запас выше точки безубыточности: {break_even_count / 100000 : .0%}')

Количество скважин в Регионе 1, имеющих запас выше точки безубыточности: 16537
Процентное соотношение скважин, имеющих запас выше точки безубыточности:  17%


In [23]:
break_even_count = data_2['product'][data_2['product'] > round(break_even)].count()
print('Количество скважин в Регионе 2, имеющих запас выше точки безубыточности:', 
      break_even_count)
print(f'Процентное соотношение скважин, имеющих запас выше точки безубыточности: {break_even_count / 100000 : .0%}')

Количество скважин в Регионе 2, имеющих запас выше точки безубыточности: 38278
Процентное соотношение скважин, имеющих запас выше точки безубыточности:  38%


Вывод:
Средний объем запасов по всем регионам меньше точки безубыточности, однако учтем тот факт, что в Регионе 0 и Регионе 1 37-38 процентов скважин имеют запас выше точки безубыточности. С учетом того, что разрабатывать будут всего 200 скважин, запасов сырья должно хватить для окупаемости проекта во всех регионах.

## Расчёт прибыли и рисков 

In [24]:
# Напишем функцию для расчёта прибыли по выбранным скважинам и предсказаниям модели

def incom (data, predictions):
    top_200 = pd.Series(predictions).sort_values(ascending=False)[:200]
    max_pred_well = pd.Series(data[top_200.index])
    return round((max_pred_well * income_per_barrel).sum() - budget_per_region)

In [25]:
print('Прибыль с 200 лучших скважин Региона 0:', incom(model_data_0[3], model_data_0[2]) / 10**6, 'млн рублей')
print()
print('Прибыль с 200 лучших скважин Региона 1:', incom(model_data_1[3], model_data_1[2]) / 10**6, 'млн рублей')
print()
print('Прибыль с 200 лучших скважин Региона 2:', incom(model_data_2[3], model_data_2[2]) / 10**6, 'млн рублей')
print()

Прибыль с 200 лучших скважин Региона 0: 3320.826043 млн рублей

Прибыль с 200 лучших скважин Региона 1: 2415.086697 млн рублей

Прибыль с 200 лучших скважин Региона 2: 2539.915946 млн рублей



In [26]:
# Создадим функцию по расчету средней прибыли, 95%-й доверительного интервала и риска убытков. 

def risks_profits(data, prediction):
     
    values = []
    for i in range(1000):
        data = data.reset_index(drop=True)
        prediction = prediction.reset_index(drop=True)
        prediction_subsample = pd.Series(prediction).sample(n=500, replace=True, random_state=state)
#         top_200 = pd.Series(data_subsample).sort_values(ascending=False)[:200]
#         max_pred_well = pd.Series(data[top_200.index])
#         values.append((max_pred_well * income_per_barrel).sum() - budget_per_region)
        values.append(incom(data, prediction_subsample))
    values = pd.Series(values) 

    mean = values.mean()
    lower = values.quantile(0.05)
    upper = values.quantile(0.95)
    risk_of_loss = (values[values < 0].count()) / (values.count())
    
        
    return mean, lower, upper, risk_of_loss

In [27]:
# Подсчитываем среднюю прибыли, 95%-й доверительного интервала и риска убытков для Региона 0

risk_region_0 = risks_profits(model_data_0[3], model_data_0[2])
print(f'Средняя прибыль: {risk_region_0[0] / 10**6 : .2f} млн руб.')
print(f'Доверительный интервал: {risk_region_0[1] / 10**6 : .2f} - {risk_region_0[2]  / 10**6: .2f} млн руб.')
print(f'Риск убытков составляет {risk_region_0[3] : .2%}')

Средняя прибыль:  394.28 млн руб.
Доверительный интервал: -10.75 -  842.51 млн руб.
Риск убытков составляет  6.10%


In [28]:
# Подсчитываем среднюю прибыли, 95%-й доверительного интервала и риска убытков для Региона 1

risk_region_1 = risks_profits(model_data_1[3], model_data_1[2])
print(f'Средняя прибыль: {risk_region_1[0] / 10**6 : .2f} млн руб.')
print(f'Доверительный интервал: {risk_region_1[1] / 10**6  : .2f} - {risk_region_1[2]  / 10**6 : .2f} млн руб.')
print(f'Риск убытков составляет {risk_region_1[3] : .2%}')

Средняя прибыль:  454.74 млн руб.
Доверительный интервал:  111.60 -  793.18 млн руб.
Риск убытков составляет  0.70%


In [29]:
# Подсчитываем среднюю прибыли, 95%-й доверительного интервала и риска убытков для Региона 2

risk_region_2 = risks_profits(model_data_2[3], model_data_2[2])
print(f'Средняя прибыль: {risk_region_2[0] / 10**6 : .2f} млн руб.')
print(f'Доверительный интервал: {risk_region_2[1] / 10**6 : .2f} - {risk_region_2[2]  / 10**6 : .2f} млн руб.')
print(f'Риск убытков составляет {risk_region_2[3] : .2%}')

Средняя прибыль:  353.66 млн руб.
Доверительный интервал: -57.42 -  764.14 млн руб.
Риск убытков составляет  7.60%


Общий вывод по проекту:\
Получены и подготовлены данные для анализа.\
Подготовлена, обучена и проверена модель.\
Определена точка безубыточности для разработки скважин.\

По результатам проведенного исследования сформированы следующие рекомендации:\
Во всех регионах средняя концентрация продукта слишком низкая, чтобы выбирать точки случайно, поэтому нужен хороший механизм для отбора.\
Регион 0 и Регион 2 не подходят для разработки скважин: регионы показывают высокие риски - выше целевого значения. \
Регион 1 подходит для разработки скважин - скважины в среднем показывают прибыль, риски ниже целевого значения.\
Точкой безубыточности по представленным данным является 111 тыс.баррелей. \
По результатам исследования рекомендуем сосредоточиться на разработке скважин в Регионе 1. \
Однако нужно иметь в виду, что 37-38 процентов скважин в Регионах 0 и 2, и 17 процентов - в Регионе 1, имеют запас выше точки безубыточности. С учетом того, что разрабатывать будут всего 200 скважин, запасов сырья должно хватить для окупаемости проекта во всех регионах.